In [1]:
import os
os.chdir("/home/yangkai/codegen-detection/")

In [2]:
from simcse.simcse import RobertaForCL
from transformers import AutoTokenizer
import torch
from tqdm import tqdm

bsz = 32
device = 2
model_args = {'pooler_type':'cls','temp':0.1,'mlp_only_train':True}
model = RobertaForCL.from_pretrained("simcse/ckpt_gcb/best_model",**model_args)
# model = RobertaForCL.from_pretrained("simcse/ckpt_gcb4_csn_pyjava/best_model",**model_args)


# model_args = {'pooler_type':'avg','temp':0.1,'mlp_only_train':True}
# model = RobertaForCL.from_pretrained("microsoft/graphcodebert-base",**model_args)

tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
# tokenizer = AutoTokenizer.from_pretrained("microsoft/unixcoder-base")

model = model.to(device)

@torch.inference_mode()
def pipeline(texts):
    inputs = tokenizer(texts,return_tensors='pt',padding=True,add_special_tokens=True,truncation=True)
    inputs = {k:v.to(device) for k,v in inputs.items()}
    sent_embed = model(**inputs,sent_emb=True).pooler_output

    return sent_embed

@torch.inference_mode()
def batch_encode(texts):
    embeds = []
    for i in tqdm(range(0,len(texts),bsz)):
        chunk = texts[i:i+bsz]
        embeds.append(pipeline(chunk))
    
    return torch.cat(embeds,dim=0)

def get_sim(data,resample_keys,orig_key):
    orig_embed = batch_encode(data[orig_key].tolist())
    resample_embeds = []
    for key in resample_keys:
        resample_embeds.append(batch_encode(data[key].tolist()))
    
    all_sim = []
    normed_orig_embed = torch.nn.functional.normalize(orig_embed,p=2,dim=-1)
    for embed in resample_embeds:
        normed_embed = torch.nn.functional.normalize(embed,p=2,dim=-1)
        sim = (normed_orig_embed * normed_embed).sum(-1)
        all_sim.append(sim)
    
    return all_sim

/home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so'), PosixPath('/home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


[2023-12-07 16:09:12,674] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
{'pooler_type': 'cls', 'temp': 0.1, 'mlp_only_train': True}


In [3]:
dset = "apps"
gen_model = ""
hum_pth = f"dataset/{dset}_hum_test.pkl"
gen_pth = f"dataset/{dset}_gen_test{gen_model}.pkl"
hum_perturb_pth = f"dataset/{dset}_hum_test_perturb_100.pkl"
gen_perturb_pth = f"dataset/{dset}_gen_test{gen_model}_perturb_100.pkl"

In [6]:
import pandas as pd
def data_prepare(orig_pth,perturb_pth):
    orig_df = pd.read_pickle(orig_pth)
    perturb_df = pd.read_pickle(perturb_pth)

    orig_funcs = []
    perturb_funcs = [[] for _ in range(16)]
    for pid,orig_func in zip(orig_df['problem_id'],orig_df['extracted_full_func']):
        orig_funcs.append(orig_func)
        perturbs = perturb_df[perturb_df.pid == pid]["extracted_full_func"].to_list()[:16]
        # print(len(perturbs))
        for i in range(16):
            try:
                perturb_funcs[i].append(perturbs[i])
            except:
                perturb_funcs[i].append("")


    new_df = orig_df[['extracted_full_func']]
    for i in range(len(perturb_funcs)):
        new_df[f'perturb_{i}'] = perturb_funcs[i]        
    
    return new_df

hum_df = data_prepare(hum_pth,hum_perturb_pth)
gen_df = data_prepare(gen_pth,gen_perturb_pth)

/tmp/ipykernel_1059044/1495082853.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'perturb_{i}'] = perturb_funcs[i]
/tmp/ipykernel_1059044/1495082853.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'perturb_{i}'] = perturb_funcs[i]
/tmp/ipykernel_1059044/1495082853.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [8]:
gen_sim = get_sim(gen_df,[f'perturb_{i}' for i in range(16)],'extracted_full_func')
hum_sim = get_sim(hum_df,[f'perturb_{i}' for i in range(16)],'extracted_full_func')

100%|██████████| 25/25 [00:12<00:00,  2.06it/s]


In [10]:
from sklearn.metrics import roc_auc_score
all_sim = [torch.concat([sim1,sim2]) for sim1,sim2 in zip(gen_sim,hum_sim)]
label = [1] * len(gen_df) + [0] * len(hum_df)

roc_aucs = []
for m in range(1,len(all_sim) + 1):
    score = sum([all_sim[i] for i in range(m)]) / m
    # score = (all_sim[0] + all_sim[1]) / 2

    score = score.tolist()

    roc_auc = roc_auc_score(label,score)
    roc_aucs.append(roc_auc)
    if m in [2,4,8,16,32]:
        print(m,roc_auc)

2 0.5349671108112668
4 0.554185360094451
8 0.5667422836903357
16 0.5705844155844155
